In [28]:
try:
    passes += 1
except NameError:
    %cd ..
    passes = 1

In [29]:
import pandas as pd

In [30]:
df = pd.read_csv("data/titanic_kaggle.csv")
df_titanic = df.sample(frac=0.75)
df_test = df.loc[df.index.difference(df_titanic), :]

In [31]:
from binarybeech.binarybeech import *
import binarybeech.utils as utils

In [32]:
import logging

logging.basicConfig(level=logging.INFO)

In [33]:
rf_titanic = RandomForest(
    df=df_titanic.copy(),
    y_name="Survived",
    cart_settings={"max_depth": 4, "min_leaf_samples": 2, "min_split_samples": 2},
    sample_frac=0.3,
    n_attributes=5,
    method="classification",
)

In [34]:
rf_titanic.train(20)

A tree with 14 leafs was created
A tree with 11 leafs was created
A tree with 12 leafs was created
A tree with 10 leafs was created
A tree with 12 leafs was created
A tree with 10 leafs was created
A tree with 14 leafs was created
A tree with 9 leafs was created
A tree with 14 leafs was created
A tree with 15 leafs was created
A tree with 11 leafs was created
A tree with 10 leafs was created
A tree with 11 leafs was created
A tree with 12 leafs was created
A tree with 15 leafs was created
A tree with 12 leafs was created
A tree with 12 leafs was created
A tree with 10 leafs was created
A tree with 11 leafs was created
A tree with 10 leafs was created


In [35]:
rf_titanic.validate_oob()

{'precision': array([0.95652174, 0.45275591]),
 'recall': array([0.74018692, 0.86466165]),
 'F-score': 0.7144389715272161,
 'accuracy': 0.7649700598802395}

In [36]:
rf_titanic.validate()

{'precision': array([0.97584541, 0.43700787]),
 'recall': array([0.73857404, 0.91735537]),
 'F-score': 0.7163954214360042,
 'accuracy': 0.7709580838323353}

In [37]:
rf_titanic.variable_importance()

{'Sex': 1.0,
 'PassengerId': 0.16615384615384615,
 'Age': 0.16000000000000017,
 'Fare': 0.16000000000000003,
 'Pclass': 0.12307692307692307,
 'Embarked': 0.07076923076923076,
 'SibSp': 0.06461538461538463,
 'Parch': 0.03384615384615391,
 'Name': 0.0,
 'Ticket': 0.0,
 'Cabin': 0.0}

In [38]:
import binarybeech.visualize as viz

viz.print_bars(rf_titanic.variable_importance(), max_width=50)

Sex       |############################### 1.0
PassengerId|#####                          0.17
Age       |#####                          0.16
Fare      |#####                          0.16
Pclass    |####                           0.12
Embarked  |##                             0.071
SibSp     |##                             0.065
Parch     |#                              0.034
Name      |                                0.0
Ticket    |                                0.0
Cabin     |                                0.0


In [39]:
rf_titanic.validate(df_test)

{'precision': array([0.9726776 , 0.45614035]),
 'recall': array([0.74166667, 0.9122807 ]),
 'F-score': 0.7248973497573722,
 'accuracy': 0.7744107744107744}

In [40]:
gbt_titanic = GradientBoostedTree(
    df=df_titanic.copy(),
    y_name="Survived",
    cart_settings={"max_depth": 2, "min_leaf_samples": 10, "min_split_samples": 10},
    sample_frac=0.75,
    n_attributes=9,
)

In [41]:
gbt_titanic.X_names

['PassengerId',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [42]:
it = gbt_titanic._initial_tree()

A tree with 1 leafs was created


In [43]:
viz.print_tree(it.tree)

0.6197604790419161



In [44]:
it.tree.root

In [45]:
gbt_titanic.learning_rate = 0.01
# gbt_titanic.create_trees(200)

In [46]:
gbt_titanic.predict(df_titanic.iloc[[0]])

array([0.61976048])

In [47]:
gbt_titanic.validate(df_test)

/home/armin/git/binarybeech/binarybeech/metrics.py:34: RuntimeWarning: invalid value encountered in true_divide
  return np.diag(m) / np.sum(m, axis=0)


{'precision': array([0., 1.]),
 'recall': array([       nan, 0.38383838]),
 'F-score': nan,
 'accuracy': 0.3838383838383838}

#k-fold cross-validation

In [48]:
c = CART(df=df_titanic, y_name="Survived", method="classification")

In [49]:
c.train(slack=1e-3)

A tree with 107 leafs was created
A tree with 90 leafs was created
A tree with 112 leafs was created
A tree with 88 leafs was created
A tree with 83 leafs was created
A tree with 88 leafs was created


INFO:binarybeech.binarybeech:beta_best: 0.0021170861712172077


A tree with 107 leafs was created


In [50]:
c.tree.leaf_count()

15

In [51]:
viz.print_tree(c.tree)

AttributeError: module 'binarybeech.utils' has no attribute 'print_tree'

In [ ]:
c.validate(df_test)

{'precision': array([0.89253188, 0.69883041]),
 'recall': array([0.82630691, 0.80201342]),
 'F-score': 0.8025093038528897,
 'accuracy': 0.8181818181818182}